# Base model with Bidirectional Encoder Representations from Transformers (BERT)

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import missingno as msno
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from keras.preprocessing import text
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer,BertTokenizerFast, BertForSequenceClassification
from transformers import TrainingArguments, Trainer, pipeline


2024-10-23 18:59:01.345295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prepare Dataset

Load JSON file

In [11]:
def load_json_file(filename):
    with open(filename) as f:
        file=json.load(f)
    return file   
    
filename="intents.json"
intents=load_json_file(filename)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hello', 'Hey', 'How are you', "What's up"], 'responses': ['Hi there', 'Hello', 'Hey', "I'm fine, thank you", 'Nothing much']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Take care'], 'responses': ['Goodbye', 'See you later', 'Take care']}, {'tag': 'thanks', 'patterns': ['Thank you', 'Thanks', 'Thanks a lot', 'I appreciate it'], 'responses': ["You're welcome", 'No problem', 'Glad I could help']}, {'tag': 'about', 'patterns': ['What can you do', 'Who are you', 'What are you', 'What is your purpose'], 'responses': ['I am a chatbot', 'My purpose is to assist you', 'I can answer questions and provide assistance']}, {'tag': 'help', 'patterns': ['Help', 'I need help', 'Can you help me', 'What should I do'], 'responses': ['Sure, what do you need help with?', "I'm here to help. What's the problem?", 'How can I assist you?']}, {'tag': 'age', 'patterns': ['How old are you', "What's your age"], 'responses': ["I don't hav

Create dataframe for patterns and tags

In [12]:
def create_df():
    df=pd.DataFrame({'Pattern':[],'Tag':[]})
    return df
    
df=create_df()
df

,Pattern,Tag


Extract data from file

In [13]:
def extract_json_info(json_file,df):
    for intent in json_file['intents']:
        for pattern in intent['patterns']:
            sentence_tag=[pattern,intent['tag']]
            df.loc[len(df.index)] = sentence_tag
    return df        
df=extract_json_info(intents,df)

## Explore data

In [14]:
df.head()

,Pattern,Tag
0,Hi,greeting
1,Hello,greeting
2,Hey,greeting
3,How are you,greeting
4,What's up,greeting


In [15]:
df.shape

(850, 2)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 850 entries, 0 to 849
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pattern  850 non-null    object
 1   Tag      850 non-null    object
dtypes: object(2)
memory usage: 19.9+ KB


In [17]:
x=len(df['Tag'].unique())
print(f"{x} classes in or tags")

print(df['Tag'].unique())

278 classes in or tags
['greeting' 'goodbye' 'thanks' 'about' 'help' 'age' 'weather' 'budget'
 'credit_score' 'name' 'favorite_color' 'hobby' 'time' 'joke' 'food'
 'movies' 'technology' 'compliment' 'meaning_of_life' 'sports' 'pets'
 'travel' 'books' 'education' 'health' 'coding' 'art' 'career'
 'technology_help' 'history' 'music' 'exercise' 'mindfulness' 'science'
 'gaming' 'positivity' 'cooking' 'relationship' 'nature' 'productivity'
 'travel_tips' 'languages' 'inspiration' 'finance_tips'
 'artificial_intelligence' 'motivation' 'future' 'movies_series'
 'self_improvement' 'robotics' 'philosophy' 'coding_languages'
 'virtual_reality' 'space_exploration' 'emotional_intelligence'
 'cybersecurity' 'creativity' 'futuristic_technology' 'entrepreneurship'
 'internet_of_things' 'universe' 'social_media' 'cuisine' 'happiness'
 'self_care' 'augmented_reality' 'global_warming' 'data_privacy'
 'positivity_quotes' 'virtual_assistant' 'emerging_technologies'
 'philanthropy' 'sustainability' 'gene_

In [18]:
df.isna().sum()

Pattern    0
Tag        0
dtype: int64

Understand the corpus

In [19]:
def get_corpus(series):
    words = []
    for text in series:
        for word in text.split():
            words.append(word.strip())
    return words

corpus = get_corpus(df.Pattern)
corpus[:5]
# print(corpus)

['Hi', 'Hello', 'Hey', 'How', 'are']

In [20]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
print(most_common)
most_common = dict(most_common)
most_common

[('me', 198), ('Tell', 188), ('about', 184), ('to', 131), ('How', 121), ('of', 121), ('a', 99), ('the', 68), ('and', 67), ('in', 49)]


{'me': 198,
 'Tell': 188,
 'about': 184,
 'to': 131,
 'How': 121,
 'of': 121,
 'a': 99,
 'the': 68,
 'and': 67,
 'in': 49}

Create another dataframe for patterns, tags and labels

In [21]:
df2 = df.copy()
df2.head()

,Pattern,Tag
0,Hi,greeting
1,Hello,greeting
2,Hey,greeting
3,How are you,greeting
4,What's up,greeting


Create list of unique tags as labels

In [22]:
labels=df2['Tag'].unique().tolist()
labels=[s.strip() for s in labels]
labels[:5]

['greeting', 'goodbye', 'thanks', 'about', 'help']

Create `id` to `label` and vise versa

In [23]:
id2label={id:label for id,label in enumerate(labels)}
label2id={label:id for id,label in enumerate(labels)}
num_labels=len(labels)

In [24]:
id2label

{0: 'greeting',
 1: 'goodbye',
 2: 'thanks',
 3: 'about',
 4: 'help',
 5: 'age',
 6: 'weather',
 7: 'budget',
 8: 'credit_score',
 9: 'name',
 10: 'favorite_color',
 11: 'hobby',
 12: 'time',
 13: 'joke',
 14: 'food',
 15: 'movies',
 16: 'technology',
 17: 'compliment',
 18: 'meaning_of_life',
 19: 'sports',
 20: 'pets',
 21: 'travel',
 22: 'books',
 23: 'education',
 24: 'health',
 25: 'coding',
 26: 'art',
 27: 'career',
 28: 'technology_help',
 29: 'history',
 30: 'music',
 31: 'exercise',
 32: 'mindfulness',
 33: 'science',
 34: 'gaming',
 35: 'positivity',
 36: 'cooking',
 37: 'relationship',
 38: 'nature',
 39: 'productivity',
 40: 'travel_tips',
 41: 'languages',
 42: 'inspiration',
 43: 'finance_tips',
 44: 'artificial_intelligence',
 45: 'motivation',
 46: 'future',
 47: 'movies_series',
 48: 'self_improvement',
 49: 'robotics',
 50: 'philosophy',
 51: 'coding_languages',
 52: 'virtual_reality',
 53: 'space_exploration',
 54: 'emotional_intelligence',
 55: 'cybersecurity',
 56

In [25]:
label2id

{'greeting': 0,
 'goodbye': 1,
 'thanks': 2,
 'about': 3,
 'help': 4,
 'age': 5,
 'weather': 6,
 'budget': 7,
 'credit_score': 8,
 'name': 9,
 'favorite_color': 10,
 'hobby': 11,
 'time': 12,
 'joke': 13,
 'food': 14,
 'movies': 15,
 'technology': 16,
 'compliment': 17,
 'meaning_of_life': 18,
 'sports': 19,
 'pets': 20,
 'travel': 21,
 'books': 22,
 'education': 23,
 'health': 24,
 'coding': 25,
 'art': 26,
 'career': 27,
 'technology_help': 28,
 'history': 29,
 'music': 30,
 'exercise': 31,
 'mindfulness': 32,
 'science': 33,
 'gaming': 34,
 'positivity': 35,
 'cooking': 36,
 'relationship': 37,
 'nature': 38,
 'productivity': 39,
 'travel_tips': 40,
 'languages': 41,
 'inspiration': 42,
 'finance_tips': 43,
 'artificial_intelligence': 44,
 'motivation': 45,
 'future': 46,
 'movies_series': 47,
 'self_improvement': 48,
 'robotics': 49,
 'philosophy': 50,
 'coding_languages': 51,
 'virtual_reality': 52,
 'space_exploration': 53,
 'emotional_intelligence': 54,
 'cybersecurity': 55,
 'c

Add converted `label2id` to df2

In [26]:
df2['labels'] = df2['Tag'].map(lambda x: label2id[x.strip()])
df2.head()

,Pattern,Tag,labels
0,Hi,greeting,0
1,Hello,greeting,0
2,Hey,greeting,0
3,How are you,greeting,0
4,What's up,greeting,0


In [27]:
X=list(df2['Pattern'])
X[:5]

['Hi', 'Hello', 'Hey', 'How are you', "What's up"]

In [28]:
y=list(df2['labels'])
y[:5]

[0, 0, 0, 0, 0]

Define training and test datasets

In [29]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=100)

Define base model

In [30]:
model_name="bert-base-uncased"
max_len=256

Load tokenizer

In [31]:
tokenizer=BertTokenizer.from_pretrained(model_name,max_length=max_len)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Load model

In [32]:
model=BertForSequenceClassification.from_pretrained(model_name,num_labels=num_labels,id2label=id2label,label2id=label2id)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Define encoders

In [33]:
train_encoding=tokenizer(X_train,truncation=True,padding=True)
test_encoding=tokenizer(X_test,truncation=True,padding=True)

In [34]:
full_data = tokenizer(X, truncation=True, padding=True)

Data loader

In [35]:
class DataLoader(Dataset):
    
    def __init__(self, encodings, labels):
        
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
               
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [36]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)

In [37]:
fullDataLoader = DataLoader(full_data, y_test)

In [38]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids  # True labels
    preds = pred.predictions.argmax(-1)  # Predicted labels (argmax along the last axis)

    # Using precision_recall_fscore_support to compute precision, recall, F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    # Using accuracy_score to compute accuracy
    acc = accuracy_score(labels, preds)
    
    # Returning a dictionary containing the computed metrics
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }


Define training arguments

In [39]:
training_args = TrainingArguments(
    output_dir='./output', 
    do_train=True,
    do_eval=True,
    num_train_epochs=100,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=16,
    warmup_steps=100,                
    weight_decay=0.05,
    logging_strategy='steps',
    logging_dir='./logs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    load_best_model_at_end=True
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
trainer = Trainer(
    model=model,
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=test_dataloader,            
    compute_metrics= compute_metrics
)

In [41]:
%%time
trainer.train()

  0%|          | 0/2000 [00:00<?, ?it/s]

{'loss': 5.6529, 'grad_norm': 4.828703880310059, 'learning_rate': 2.5e-05, 'epoch': 2.5}


  0%|          | 0/14 [00:00<?, ?it/s]

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 5.6662702560424805, 'eval_Accuracy': 0.004694835680751174, 'eval_F1': 0.0006303183107469272, 'eval_Precision': 0.00033266799733865603, 'eval_Recall': 0.005988023952095809, 'eval_runtime': 1.2009, 'eval_samples_per_second': 177.361, 'eval_steps_per_second': 11.658, 'epoch': 2.5}
{'loss': 5.499, 'grad_norm': 5.296258449554443, 'learning_rate': 5e-05, 'epoch': 5.0}


...

{'eval_loss': 3.233506679534912, 'eval_Accuracy': 0.37089201877934275, 'eval_F1': 0.2928571428571428, 'eval_Precision': 0.29642857142857143, 'eval_Recall': 0.3142857142857143, 'eval_runtime': 0.591, 'eval_samples_per_second': 360.412, 'eval_steps_per_second': 23.689, 'epoch': 97.5}
{'loss': 0.0526, 'grad_norm': 0.23745191097259521, 'learning_rate': 0.0, 'epoch': 100.0}


  0%|          | 0/14 [00:00<?, ?it/s]

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 3.2338368892669678, 'eval_Accuracy': 0.37089201877934275, 'eval_F1': 0.2928571428571428, 'eval_Precision': 0.29642857142857143, 'eval_Recall': 0.3142857142857143, 'eval_runtime': 0.6177, 'eval_samples_per_second': 344.843, 'eval_steps_per_second': 22.666, 'epoch': 100.0}
{'train_runtime': 830.2487, 'train_samples_per_second': 76.724, 'train_steps_per_second': 2.409, 'train_loss': 1.1354074995517731, 'epoch': 100.0}
CPU times: user 12min 5s, sys: 48.4 s, total: 12min 53s
Wall time: 13min 50s


TrainOutput(global_step=2000, training_loss=1.1354074995517731, metrics={'train_runtime': 830.2487, 'train_samples_per_second': 76.724, 'train_steps_per_second': 2.409, 'total_flos': 459421689199200.0, 'train_loss': 1.1354074995517731, 'epoch': 100.0})

Evaluate trainer

In [42]:
q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]
pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.036079,0.998430,0.998384,0.999273,0.998182
test,3.233837,0.370892,0.292857,0.296429,0.314286


Save model and tokenizer

In [43]:
model_path = "base"

In [44]:
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('base/tokenizer_config.json',
 'base/special_tokens_map.json',
 'base/vocab.txt',
 'base/added_tokens.json')

Load model and tokenizer

In [50]:
new_model = BertForSequenceClassification.from_pretrained(model_path)
new_tokenizer= BertTokenizerFast.from_pretrained(model_path)

In [46]:
pipe= pipeline("sentiment-analysis", model=new_model, tokenizer=new_tokenizer, device='mps')

In [47]:
pipe("Hello")

[{'label': 'greeting', 'score': 0.9877205491065979}]

## Predict

In [48]:
def predict(text):
    # Tokenize the input text and convert it to PyTorch tensors
    inputs = new_tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cpu")

    # Pass the input tensors through the model
    outputs = new_model(**inputs)

    # Softmax the output to get probabilities
    probs = outputs[0].softmax(1)

    # Get the predicted label index
    pred_label_idx = probs.argmax()

    # Convert the predicted label index to the actual label using model configuration
    pred_label = new_model.config.id2label[pred_label_idx.item()]

    return pred_label


In [51]:
text = "Hello"
predict(text)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


'greeting'

In [52]:
def model_response(text):
    response = ''
    score = pipe(text)[0]['score']
    tag = pipe(text)[0]['label']
    if score < 0.8:
        response += "Don't understand"
    else:
        label = new_model.config.label2id[pipe(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])
    return response, tag

In [58]:
model_response('hi')

('Hello', 'greeting')